Задание

1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
2. Сформировать выводы по анализу.

In [20]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

In [21]:
# получим количество зарегистрировавшихся пользователей registered_users_count за май и июнь 2019 года 
# в разрезе registration_date — дней регистрации

def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count = pd.DataFrame(getRegisteredUsersCount())

In [22]:
# преобразуем колонку registration_date в формат datatime

registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

In [23]:
#  Получим количество активных пользователей active_users_count по дням с указанием даты регистрации

def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

In [24]:
#  Преобразуем колонки activity_date и registration_date в даты

for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

In [25]:
# Объединим датафреймы registered_users_count и active_users_count_with_cohorts в retention_table

retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-05-01,2019-05-01,2590,6516
1,2019-05-02,2019-05-01,2146,6516
2,2019-05-02,2019-05-02,2049,5309
3,2019-05-03,2019-05-01,1637,6516
4,2019-05-03,2019-05-02,1817,5309


In [26]:
# Посчитаем retention_rate

retention_table['retention_rate'] = retention_table['active_users_count'] / retention_table['registered_users_count']
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу

In [27]:
# Получим данные о выручке

def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
revenue = pd.DataFrame(getRevenue())

In [28]:
#  Преобразуем колонки activity_date и registration_date в даты

for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

In [29]:
# Для дальнейшего объединения датафреймов в датафрейме revenue изменим название колонки date на activity_date

revenue = revenue.rename(columns={'date':'activity_date'})

In [30]:
# Объединим датафреймы retention_table и revenue в retention_table_with_revenue

retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
retention_table_with_revenue.head()


,activity_date,registration_date,active_users_count,registered_users_count,retention_rate,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0.397483,0,1672.0,2357.590
1,2019-05-02,2019-05-01,2146,6516,0.329343,1,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,0.385948,0,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,0.251228,2,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,0.342249,1,1164.0,533.608


In [31]:
# Пропущенные значения заполним нулями

for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)

In [32]:
# Преобразуем тип в колонке users_count_with_revenue в целочисленный

retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,retention_rate,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0.397483,0,1672,2357.590
1,2019-05-02,2019-05-01,2146,6516,0.329343,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0.385948,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,0.251228,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,0.342249,1,1164,533.608


In [33]:
# Рассчитаем Retention Rate седьмого дня за май 2019

retention_may = retention_table_with_revenue[retention_table_with_revenue['registration_date']>='2019-05-01']
retention_may = retention_may[retention_may['registration_date']<'2019-06-01']
retention_may_7 = retention_may[retention_may['lifetime']==7]

# Рассчитываем Retention Rate за май 2019

print(((retention_may_7['active_users_count'].sum())*100 / retention_may_7['registered_users_count'].sum()))

12.933000957960866


In [34]:
# Рассчитаем Retention Rate седьмого дня за июнь 2019

retention_june = retention_table_with_revenue[retention_table_with_revenue['registration_date']>='2019-06-01']
retention_june = retention_june[retention_june['registration_date']<'2019-07-01']
retention_june_7 = retention_june[retention_june['lifetime']==7]

# Рассчитываем Retention Rate за июнь 2019

print(((retention_june_7['active_users_count'].sum())*100 / retention_june_7['registered_users_count'].sum()))

12.560867530346634


In [35]:
# Рассчитаем ARPPU за май 2019

print(retention_may['revenue'].sum() /retention_may['users_count_with_revenue'].sum())

0.3061246038517031


In [36]:
# Рассчитаем ARPPU за июнь 2019

print(retention_june['revenue'].sum() /retention_june['users_count_with_revenue'].sum())

0.39319397180047666


Выводы:

1. Были посчитаны показатели Retention Rate седьмого дня: на 7ое мая он составил 12,93%, на 7ое июня 12,56%.
    Средняя выручка (ARPPU) с активного пользователя в мае составила 0,30 у.е, в июне - 0,39 у.е.
2. В июне по сравнению с маем коэффециент удержания немного снизился, при этом средняя выручка с активного пользователя выросла. Исходя из полученных данных можно сделать вывод, что после внедрения новой фичи показатели практически не упали и как следствие, она вполне имеет право на жизнь. Для более детальной проработки необходимо провести дополнительный анализ.
